In [ ]:
%matplotlib inline

from matplotlib import pyplot as plt
import tensorflow as tf
import numpy as np
import gpflow

def catch_exception(func, *args, **kwargs):
    try:
        return func(*args, **kwargs)
    except Exception as e:
        return e

# <center><b>Towards GPflow 1.0</b></center>


<center><p>Artem Artemev</center>
<center><p>Cambridge, UK</center>


### Agenda


* Use TensorFlow "right".

* GPflow before.

* GPflow re-design.

* Code examples.

* What's next?

# <center><b>"Simplicity is prerequisite for reliability."</b></center>

_<center><p><p>-Edsger Dijkstra</center>_

# <center><b>Use TensorFlow "right"</b></center>

### TensorFlow has two major terms

* Graph

* Session

### Graph

* It is only model definition... ?!
* Model's symbolic representation via operations and tensors.

* Graph is **immutable**. (1)
* Graph is a set of scopes - operations and variables.

* From (1) it follows that scopes are not modifiable too.

### Example

In [ ]:
a = tf.get_variable('a', shape=())
b = tf.get_variable('b', shape=())

with tf.name_scope('c'):
    c1 = a + b

with tf.name_scope('c'):
    c2 = a + b

In [ ]:
c1.name

In [ ]:
c2.name

### Session

* Works with only graph.
* Executes graph operations.
* Allocates resources, not the **graph**.

### One model - one graph - many sessions !

* Only **graph definition** matters in GPflow design.

* **Session** can be interchangeable.

# <center><b>GPflow before</b></center>

### Status Quo

* One _flat_ TensorFlow variable contains all parameters.

* TensorFlow parameter name scope is _virtual_.

* Parameter as a FSM (finite state machine). For e.g. parameter has `recompile` flag.

* **Next things deserve stand-alone slides!**

### Hacky functions

* get_free_state

* get_feed_dict_keys

* update_feed_dict

* make_tf_array

* get_free_state

### Hacky functions

* Most of the code exist just to make this possible:
```python
def obj(x):
    self.num_fevals += 1
    feed_dict = {self._free_vars: x}
    self.update_feed_dict(self._feed_dict_keys, feed_dict)
    f, g = self.session.run([self._minusF, self._minusG], feed_dict=feed_dict)
    return f.astype(np.float64), g.astype(np.float64)
```

### Not clear interface

* When user should call `compile`?
* What autoflow does with model internals?

# <center><b>GPflow re-design</b></center>

### Main goals

* Use external graphs and sessions.

* One param - one TensorFlow variable.

* Optimization is not part of the model.

* Simple and clean design (as far as possible).

### Glossary

* **Build** is a process of defining the _graph_.

* **Compile** is a two phase process which consist of building and initialising for specific _session_. 

### Change

* _Build_ status enum class: `YES`, `NO`, `NOT_COMPATIBLE_GRAPH`

* _ICompilable_ interface.

* _CompilableNode_ class joins two separate interfaces _ICompilable_ and _Parentable_. It implements common parts, provides useful methods.

* _AutoFlow_ class. It is not decorator anymore! The _autoflow_ decorator replaces it.

* *param_as_tensor* decorator. _Explicit_ convertion from parameter to tensor.

* _GPflowError_ exception.

### Compilable interface

```python
class ICompilable:
    __metaclass__ = abc.ABCMeta

    @abc.abstractproperty
    def graph(self):
        pass

    @abc.abstractmethod
    def compile(self, session=None, keep_session=False):
        pass

    @abc.abstractmethod
    def initialize(self, session=None):
        pass

    @abc.abstractmethod
    def is_built(self, graph=None):
        pass

    @abc.abstractmethod
    def _build(self):
        pass

    @abc.abstractmethod
    def clear(self):
        pass
```

# <center><b>Examples</b></center>

### Parameter. _Create_

In [ ]:
tf.reset_default_graph()

In [ ]:
a = gpflow.Param(10, name='a_param', trainable=False)
a.compile()
print("Variable tensor: '{0}'".format(a.var_tensor))
print("Trasformed tensor: '{0}'".format(a.transformed_tensor))
print("Prior tensor: '{0}'".format(a.prior_tensor))

In [ ]:
tf.reset_default_graph()

In [ ]:
tf.reset_default_graph()
tensor = tf.get_variable('b_param', shape=())
b = gpflow.Param(tensor, name='b_param')
b.compile()
b.var_tensor is tensor

In [ ]:
tf.reset_default_graph()

In [ ]:
tensor = tf.get_variable('c_param/variable', shape=())
c = gpflow.Param(10, name='c_param')
c.compile()
c.var_tensor is tensor

### Parameter. _Create_

In [ ]:
tf.reset_default_graph()

In [ ]:
tensor = tf.get_variable('c_param/variable', shape=())
c = gpflow.Param(10, name='c_param', trainable=False)
catch_exception(lambda: c.compile())

### Parameter. _Methods_

In [ ]:
tf.reset_default_graph()

In [ ]:
a = gpflow.Param(10, name='c_param')
a.compile()
a.read_value()

In [ ]:
a.assign(11) # NOTE: temporary assignement for specific session.
a.read_value()

In [ ]:
a.initialize()
a.read_value()

### Parameter. _Graph_

In [ ]:
tf.reset_default_graph()

In [ ]:
a = gpflow.Param(10, name='a_param')
graph = tf.Graph()
with graph.as_default():
    a.compile()
print("Default graph: {0}".format(tf.get_default_graph()))
print("Param graph: {0}".format(a.graph))

In [ ]:
a.is_built(graph)

In [ ]:
a.is_built(tf.get_default_graph())

### Parameter. _Session_

In [ ]:
tf.reset_default_graph()

In [ ]:
a = gpflow.Param(10, name='a_param')
graph = tf.Graph()
session = tf.Session(graph=graph)

In [ ]:
a.compile(session=session, keep_session=False)
a.session is None

In [ ]:
a.graph

In [ ]:
with session.as_default():
    a.compile()
a.session is session

### Parameter. _Session_

In [ ]:
tf.reset_default_graph()

In [ ]:
a = gpflow.Param(10, name='a_param')
a.compile()  # Uses default graph

session = tf.Session(graph=tf.Graph())
catch_exception(lambda: a.compile(session=session, keep_session=False))

### Parameterized objects. _Create_

In [ ]:
tf.reset_default_graph()

In [ ]:
p = gpflow.params.Parameterized()

In [ ]:
tensor = tf.get_variable('tensor', shape=(), trainable=False)
p.a = gpflow.Param(tensor, trainable=False)
p.b = gpflow.Param(10)
p.c = gpflow.DataHolder(10)
print("p.a = {0}".format(p.a.full_name))
print("p.b = {0}".format(p.b.full_name))
print("p.c = {0}".format(p.c.full_name))

In [ ]:
p.compile()
print("List of data holders: {0}".format(list(p.data_holders)))
print("List of trainable tensors: {0}".format(list(p.trainable_tensors)))
print("Prior tensor: {0}".format(p.prior_tensor))

### Parameterized objects. _Change parameters_

In [ ]:
print("p.b before assign: {0}".format(p.b.read_value()))
p.b = 1e3
print("p.b after assign: {0}".format(p.b.read_value()))

### Decorators

In [ ]:
tf.reset_default_graph()

In [ ]:
class DumbModel(gpflow.model.Model):
    def __init__(self, name=None):
        gpflow.model.Model.__init__(self, name=name)
        self.a = gpflow.Param(3.)
        
    @gpflow.autoflow()
    def execute1(self):
        return self.a.var_tensor * self.likelihood_tensor
    
    @gpflow.autoflow()
    @gpflow.params_as_tensors
    def execute2(self):
        return self.a * self.likelihood_tensor

    @gpflow.params_as_tensors
    def _build_likelihood(self):
        return -tf.square(self.a)

In [ ]:
m = DumbModel(name='simple')
m.compile()
print("Execute 1 for {name}: {result}".format(name=m.name, result=m.execute1()))
print("Execute 2 for {name}: {result}".format(name=m.name, result=m.execute2()))

### GP regression

In [ ]:
tf.reset_default_graph()

def plot_model_predic_f(m):
    xx = np.linspace(-0.1, 1.1, 100)[:,None]
    mean, var = m.predict_y(xx)
    plt.figure(figsize=(12, 6))
    plt.plot(X, Y, 'kx', mew=2)
    plt.plot(xx, mean, 'b', lw=2)
    plt.fill_between(xx[:,0], mean[:,0] - 2*np.sqrt(var[:,0]), mean[:,0] + 2*np.sqrt(var[:,0]), color='blue', alpha=0.2)
    plt.xlim(-0.1, 1.1)

In [ ]:
N = 10
X = np.random.rand(N, 1)
Y = np.sin(12 * X) + 0.1 * np.cos(2.5 * X) + np.random.randn(N, 1) * 0.9 + 3

k = gpflow.kernels.Matern52(1, lengthscales=0.2)
m = gpflow.model.GPR(X, Y, kern=k)
m.likelihood.variance = 0.1

In [ ]:
catch_exception(lambda: m.predict_f())

### GP regression

In [ ]:
m.compile()
plot_model_predic_f(m)